In [ ]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import argparse
from utils.utilities import int16_to_float32, move_data_to_device, set_labels

import h5py
import pandas as pd

import numpy as np
import librosa
import librosa.display
from scipy.stats import norm

import mlflow
import torch

import matplotlib.pyplot as plt
import seaborn as sns

from captum.attr import Deconvolution, GuidedBackprop, NeuronDeconvolution, NeuronGuidedBackprop

## Settings and Utils

In [ ]:
# Adjust according to your experiment
ref_fold = "8"
run_id = ""
tracking_server = ""
workspace_file = ""
dataset_dir = ""
device = 'cuda' if torch.cuda.is_available() else 'cpu'

mlflow.set_tracking_uri(f"{tracking_server}:5000")
logged_model = mlflow.pytorch.load_model(f"runs:/{run_id}/models")
logged_model = logged_model.eval()

In [ ]:
with h5py.File(workspace_file, 'r') as h5_file:
    folds = h5_file['fold'][:].astype(np.float32)

    indexes = np.where(folds == int(ref_fold))[0]
    inp_data = int16_to_float32(h5_file["waveform"][indexes])
    inp_data = move_data_to_device(inp_data, device)
    labels = h5_file["target"][indexes]
    labels = move_data_to_device(labels, device)

_, lb_to_idx = set_labels(dataset_dir)
idx_to_label = {idx: label for label, idx in lb_to_idx.items()}
target = [idx for label, idx in lb_to_idx.items() if label.startswith("albilora")]
# allows gradient for the method
inp_data.requires_grad_()

In [ ]:
# Auxiliar method to plot the attributions
def plot_frame_attributions(attributions, title="Average Frames importance using Deconvolution"):
    plt.figure(figsize=(10, 6))
    plt.bar(np.arange(len(attributions[0])), np.mean(attributions.cpu().detach().numpy(), axis=0))
    plt.xlabel("Frames")
    plt.title(title)

In [ ]:
def layer_attribution(model, layer, layer_name, inp_data, neuron, algorithm="deconv", verbose=False):
    channels = neuron[0]
    time_steps = neuron[1]
    mel_bins = neuron[2]
    
    tot = 0
    iterations = channels * time_steps * mel_bins
    
    if algorithm == "deconv":
        layer_deconv = NeuronDeconvolution(model, layer)
    elif algorithm == "guided":
        layer_deconv = NeuronGuidedBackprop(model, layer)
    else:
        raise ValueError(f"Incorrect algorithm {algorithm}. Expected 'deconv' or 'guided'")
        
    out_dict = {"layer": [], "channel": [], "time_steps": [], "mel_bins": [], "frameAvgAttr": []}
    
    for channel in range(channels):
        for time_step in range(time_steps):
            for mel_bin in range(mel_bins):
                conv1_neuron_attr = layer_deconv.attribute(inp_data, (channel, time_step, mel_bin))
                out_dict["layer"].append(layer_name)
                out_dict["channel"].append(channel)
                out_dict["time_steps"].append(time_step)
                out_dict["mel_bins"].append(mel_bin)
                out_dict["frameAvgAttr"].append(np.mean(np.mean(conv1_neuron_attr.cpu().detach().numpy(), axis=0)))
                if verbose:
                    tot += 1
                    print(f"Progress: {tot}/{iterations}-----------{100*tot/iterations:.2f}%", end="\r")
                                                
    return out_dict

In [ ]:
# Compute ZCR to possible percussive audios
# https://github.com/tyiannak/pyAudioAnalysis
def zero_crossing_rate(frame):
    count = len(frame)
    count_zero = np.sum(np.abs(np.diff(np.sign(frame)))) / 2
    return np.float64(count_zero) / np.float64(count - 1.0)

In [ ]:
rng = np.random.default_rng(135)

## Zeiler and Fergus (2014)- Deconvolution

### Model attribution

In [ ]:
deconv = Deconvolution(logged_model)

Calcular atribuições para o canto e cada sílaba

In [ ]:
ca_attribution = deconv.attribute(inp_data, target[0])

In [ ]:
syl1_attribution = deconv.attribute(inp_data, target[1])

In [ ]:
syl2_attribution = deconv.attribute(inp_data, target[2])

In [ ]:
plot_frame_attributions(ca_attribution)

In [ ]:
plot_frame_attributions(syl1_attribution)

In [ ]:
plot_frame_attributions(syl2_attribution)

In [ ]:
# The detected class(es) for the first experiment with 100 iterations
others_attribution = deconv.attribute(inp_data, lb_to_idx["others"])

In [ ]:
plot_frame_attributions(others_attribution)

There was a significant impact in attribution for the middle of the audio. Maybe the way I've selected the audios has influency for greather attributions between 0.25 seconds and 0.625 seconds (5000/24000 and 15000/24000)

Checking the behavior above for background sound

In [ ]:
savanna = [idx for label, idx in lb_to_idx.items() if label.startswith("SAV")]

In [ ]:
sav_dry_attributions = deconv.attribute(inp_data, savanna[0])
plot_frame_attributions(sav_dry_attributions)

In [ ]:
sav_wet_attributions = deconv.attribute(inp_data, savanna[1])
plot_frame_attributions(sav_wet_attributions)

In [ ]:
forest = [idx for label, idx in lb_to_idx.items() if label.startswith("FOR")]

In [ ]:
for_dry_attributions = deconv.attribute(inp_data, forest[0])
plot_frame_attributions(for_dry_attributions)

In [ ]:
for_wet_attributions = deconv.attribute(inp_data, forest[1])
plot_frame_attributions(for_wet_attributions)

Is this behavior the same for individual layers?

### Neuron attribution

First Convolutional Block

In [ ]:
neuron_deconv_conv1 = NeuronDeconvolution(logged_model, logged_model.base.conv_block1)

- Neuron's indices: (0..63, 0..37, 0..31) - (channels, time_steps or num_frames, mel_bins), i.e, Neuron's output dimension
- channels always doubling
- num_frames = 1+ceil(len_y / hop_length) if center is True
- else 1 + ceil(len_y - n_fft) / hop_length where len_y is the length of the audio

In [ ]:
neuron_ca_attributions = neuron_deconv_conv1.attribute(inp_data, (0, 37, 31))

In [ ]:
plot_frame_attributions(neuron_ca_attributions, title="Average Frames importance for a Neuron on 1st Conv Block")

Again for the same block

In [ ]:
neuron_ca_attributions_2 = neuron_deconv_conv1.attribute(inp_data, (33, 15, 12))

In [ ]:
plot_frame_attributions(neuron_ca_attributions_2, title="Average Frames importance for a Neuron on 1st Conv Block")

### Layer attribution

It's too much time run the algorithm for all 77824 neurons. I will define a bootrasp distribution from some neurons to speed up processing and minimize biases on analysis

In [ ]:
# Tentative to centralize as cubic root of 360 as 7.11
samples = 360 # Approximately
channels = rng.integers(64)
time_steps = rng.integers(np.floor(samples/channels))
mel_bins = int(np.ceil(samples / (channels * time_steps)))
print(channels * time_steps * mel_bins, channels, time_steps, mel_bins)

#### First layer

In [ ]:
%%time
frames_attr_layer_1_deconv_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block1,
    "Conv 1 block",
    inp_data,
    (channels, time_steps, mel_bins), verbose=True
)

df = pd.DataFrame(frames_attr_layer_1_deconv_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap))

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

#### Second layer

In [ ]:
%%time
frames_attr_layer_2_deconv_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block2,
    "Conv 2 block",
    inp_data,
    (channels, time_steps, mel_bins), verbose=True
)

df = pd.DataFrame(frames_attr_layer_2_deconv_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap));

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

5th layer

In [ ]:
time_steps = 2
mel_bins = 2
channels = 90

In [ ]:
%%time
frames_attr_layer_5_deconv_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block5,
    "Conv 5 block",
    inp_data,
    (channels, time_steps, mel_bins),
    verbose=True
)

df = pd.DataFrame(frames_attr_layer_5_deconv_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap));

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

## Springebenberg et al. 2015- Guided Backpropagation

### Model attribution

In [ ]:
guided_backprop = GuidedBackprop(logged_model)

Call and syllables

In [ ]:
ca_attribution = guided_backprop.attribute(inp_data, target[0])

In [ ]:
syl1_attribution = guided_backprop.attribute(inp_data, target[1])

In [ ]:
syl2_attribution = guided_backprop.attribute(inp_data, target[2])

In [ ]:
plot_frame_attributions(ca_attribution)

In [ ]:
plot_frame_attributions(syl1_attribution)

In [ ]:
plot_frame_attributions(syl2_attribution)

In [ ]:
# The detected class(es) for the first experiment with 100 iterations
others_attribution = deconv.attribute(inp_data, lb_to_idx["others"])

In [ ]:
plot_frame_attributions(others_attribution)

Background attributions

In [ ]:
savanna = [idx for label, idx in lb_to_idx.items() if label.startswith("SAV")]

In [ ]:
sav_dry_attributions = deconv.attribute(inp_data, savanna[0])
plot_frame_attributions(sav_dry_attributions)

In [ ]:
sav_wet_attributions = deconv.attribute(inp_data, savanna[1])
plot_frame_attributions(sav_wet_attributions)

In [ ]:
forest = [idx for label, idx in lb_to_idx.items() if label.startswith("FOR")]

In [ ]:
for_dry_attributions = deconv.attribute(inp_data, forest[0])
plot_frame_attributions(for_dry_attributions)

In [ ]:
for_wet_attributions = deconv.attribute(inp_data, forest[1])
plot_frame_attributions(for_wet_attributions)

### Layer attribution

First layer

In [ ]:
%%time
frames_attr_layer_1_guided_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block1,
    "Conv 1 block",
    inp_data,
    (channels, time_steps, mel_bins),
    "guided",
    verbose=True
)

df = pd.DataFrame(frames_attr_layer_1_guided_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap));

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

Second layer

In [ ]:
%%time
frames_attr_layer_2_guided_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block2,
    "Conv 2 block",
    inp_data,
    (channels, time_steps, mel_bins),
    "guided",
    verbose=True
)

df = pd.DataFrame(frames_attr_layer_2_guided_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap));

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

5th layer

In [ ]:
time_steps = 2
mel_bins = 2
channels = 90

In [ ]:
%%time
frames_attr_layer_5_guided_meta = layer_attribution(
    logged_model,
    logged_model.base.conv_block5,
    "Conv 5 block",
    inp_data,
    (channels, time_steps, mel_bins),
    "guided",
    verbose=True
)

df = pd.DataFrame(frames_attr_layer_5_guided_meta)

attr_mean_bootstrap = []
for i in range(10000):
    attr_mean_bootstrap.append(df.sample(frac=1, replace=True)["frameAvgAttr"])

std_error = np.std(attr_mean_bootstrap, ddof=1)
pop_std_error = std_error * np.sqrt(samples)
print(std_error, pop_std_error)

In [ ]:
plt.hist(np.array(attr_mean_bootstrap));

90% confidence interval

In [ ]:
point_estimate = np.mean(attr_mean_bootstrap)
lower = norm.ppf(0.05, loc=point_estimate, scale=std_error)
upper = norm.ppf(0.95, loc=point_estimate, scale=std_error)
print(lower, upper)

## Feature Visualizations

Iterate over each audio considering the parameters for spectrogram generation

In [ ]:
start = 0

client = mlflow.MlflowClient()
run = client.get_run(run_id)
run_data = run.data
tags = run_data.tags

window_size = int(tags["window_size"])
hop_size = int(tags["hop_size"])
cur_window = 0
num_audios = len(inp_data)

zcr_audios = {"avgZcr": [], "label": [], "frame": []}

num_frame = 1
while cur_window + window_size - 1 < inp_data.shape[1]:

    for i in range(num_audios):
        frame = inp_data[i][cur_window:cur_window+window_size]

        frame_zcr = zero_crossing_rate(frame.cpu().detach().numpy())
        zcr_audios["avgZcr"].append(np.mean(frame_zcr))
        zcr_audios["label"].append(idx_to_label[np.argmax(labels.cpu().detach().numpy())])
        zcr_audios["frame"].append(num_frame)

    num_frame += 1
    cur_window += hop_size

In [ ]:
zcr_audios = pd.DataFrame(zcr_audios)
zcr_audios

In [ ]:
_, ax = plt.subplots()
sns.histplot(x="avgZcr", data=zcr_audios, element="step", fill=False, ax=ax);

In [ ]:
first_18_frames_zcr_audios = zcr_audios[zcr_audios["frame"] <= 18]
frames_18_36_zcr_audios = zcr_audios[(zcr_audios["frame"] > 18) & (zcr_audios["frame"] <= 36)]
frames_36_54_zcr_audios = zcr_audios[(zcr_audios["frame"] > 36) & (zcr_audios["frame"] <= 54)]
frames_54_above_zcr_audios = zcr_audios[zcr_audios["frame"] > 54]

In [ ]:
_, ax = plt.subplots()
sns.histplot(x="avgZcr", data=first_18_frames_zcr_audios, hue="frame", element="step", fill=False, ax=ax);

In [ ]:
_, ax = plt.subplots()
sns.histplot(x="avgZcr", data=frames_18_36_zcr_audios, hue="frame", element="step", fill=False, ax=ax);

In [ ]:
_, ax = plt.subplots()
sns.histplot(x="avgZcr", data=frames_36_54_zcr_audios, hue="frame", element="step", fill=False, ax=ax);

In [ ]:
_, ax = plt.subplots()
sns.histplot(x="avgZcr", data=frames_54_above_zcr_audios, hue="frame", element="step", fill=False, ax=ax);

Check 10 random spectrograms

In [ ]:
fig, axes = plt.subplots(figsize=(18, 35), nrows=10)
sr = int(tags["sample_rate"])

for i in range(10):
    audio = rng.integers(num_audios)

    librosa.display.specshow(np.abs(
        librosa.stft(inp_data[audio].cpu().detach().numpy(),
            n_fft=window_size, win_length=window_size, hop_length=hop_size, center=True)
        ),
        sr=sr, x_axis="time", y_axis="linear", hop_length=hop_size,
        fmin=int(tags["fmin"]), fmax=int(tags["fmax"]), ax=axes[i]
    )
    axes[i].set_title(f"{idx_to_label[np.argmax(labels[audio].cpu().detach().numpy())]} spectrogram",
                    {'fontsize': 11}, loc='left')
    
plt.show()

## Conclusions

- Most activities are between 0.25 seconds and 0.5 seconds
- Model gave more attention to the middle of the audio
- Low level convolutional blocks had less attribution, maybe due to more specific feature extraction from AudioSet dataset

## Next steps

- [ ] Global gradient algorithms
- [ ] Pertubation algorithm


- Approaches based on local gradient analysis
Selected period approach:
- [ ] Change duration for load each audio/hop_size

Resolution period approach:
- [ ] Change parameters for spectrogram

Model approach
- [ ] Train PANNs from scratch with our data (I believe it will decrease performance)
- [ ] Fine-tune PANNs with more iterations